In [1]:
#! /usr/bin/env python
# -*-coding:utf-8-*-
# Author: Ming Chen
# create date: 2019-11-27 10:36:59
# description: 使用HMM进行命名实体识别NER
# 包括人名PER,地名LOC机构名ORG,其他O
import numpy as np
from tqdm import tqdm

In [2]:
class HMM_BIO_method1:
    def __init__(self):
        self.n_tag = 3  # 表示所有标签个数
        self.n_char = 65535  # 所有字符的Unicode编码个数
        self.epsilon = 1e-100  # 无穷小量
        self.tag2idx = {'B-ORG': 0,
                        'I-ORG': 1,
                        'O': 2}
        self.idx2tag = dict(zip(self.tag2idx.values(), self.tag2idx.keys()))
        self.A = np.zeros((self.n_tag, self.n_tag))  # 转移概率矩阵,shape:7*7
        self.B = np.zeros((self.n_tag, self.n_char))  # 发射概率矩阵,shape:7*字的个数
        self.pi = np.zeros(self.n_tag)  # 初始隐状态概率,shape：4

    def train(self, corpus_path ,savedir=None):
        """
        函数说明： 训练HMM模型, 得到模型参数pi,A,B

        Parameter：
        ----------
            corpus_path - 语料库的位置
        Return:
        -------
            None
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 13:42:50
        """
        with open(corpus_path, mode='r', encoding='utf-8') as fr:
            line = fr.readline()
            print('开始训练数据：')
            preline="\n"
            cnt=10000000
            
            while line:
                
                #print(line)
                if len(line) == 1:
                    preline="\n"
                else:
                    cur_char,cur_tag=line[0],line[2:-1]
                    self.B[self.tag2idx[cur_tag]][ord(cur_char)] += 1
                    if len(preline) == 1:
                        self.pi[self.tag2idx[cur_tag]] += 1
                    else:
                        pre_char, pre_tag = preline[0],preline[2:-1]
                        self.A[self.tag2idx[pre_tag]][self.tag2idx[cur_tag]] += 1
                    preline=line
                line = fr.readline()
                
                cnt-=1
                if cnt==0:
                    print('write 1000,0000 rows')
                    cnt=10000000
                    
        print('参数统计结果(A,PI)')
        print(f'A:{self.A}')
        print(f'PI:{self.pi}')
        
        self.pi[self.pi == 0] = self.epsilon  # 防止数据下溢,对数据进行对数归一化
        self.pi = np.log(self.pi) - np.log(np.sum(self.pi))
        self.A[self.A == 0] = self.epsilon
        self.A = np.log(self.A) - np.log(np.sum(self.A, axis=1, keepdims=True))
        self.B[self.B == 0] = self.epsilon
        self.B = np.log(self.B) - np.log(np.sum(self.B, axis=1, keepdims=True))
        
        print('训练参数结果(A,B,PI)')
        print(f'A:{self.A}')
        print(f'B:{self.B}')
        print(f'PI:{self.pi}')
        
        self.save_Param(savedir)
    
    def save_Param(self,savedir):
        if savedir==None:
            savedir='param/BIO/method1/'
        np.savetxt(f'{savedir}A.csv',self.A,delimiter=',')
        np.savetxt(f'{savedir}B.csv',self.B,delimiter=',')
        np.savetxt(f'{savedir}PI.csv',self.pi,delimiter=',')
        print(f'训练参数已保存在 {savedir}A.csv')
        print()
    
    def load_Param(self,loaddir='param/BIO/method1/'):
        self.A=np.genfromtxt(f'{loaddir}A.csv',delimiter=',')
        self.B=np.genfromtxt(f'{loaddir}B.csv',delimiter=',')
        self.pi=np.genfromtxt(f'{loaddir}PI.csv',delimiter=',')
        print(f'训练参数(A,B,PI)已从 {loaddir}A.csv 中读取成功')
        print()
    
    
    def viterbi(self, Obs):
        """
        函数说明： 使用viterbi算法进行解码

        Parameter：
        ----------
            Obs - 要解码的文本string
        Return:
        -------
            path - 最可能的隐状态路径
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 16:52:42
        """
        T = len(Obs)
        delta = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        psi = np.zeros((T, self.n_tag))  # shape: 观测文本数量*7
        delta[0] = self.pi[:] + self.B[:, ord(Obs[0])]  # 初始化
        for i in range(1, T):
            temp = delta[i - 1].reshape(self.n_tag, -1) + self.A  # 这里运用到了矩阵的广播算法
            delta[i] = np.max(temp, axis=0)
            delta[i] = delta[i, :] + self.B[:, ord(Obs[i])]
            psi[i] = np.argmax(temp, axis=0)
        path = np.zeros(T)
        path[T - 1] = np.argmax(delta[T - 1])
        for i in range(T - 2, -1, -1):  # 回溯
            path[i] = int(psi[i + 1][int(path[i + 1])])
        return path

    def predict(self, Obs):
        """
        函数说明： 将文本进行命名实体识别

        Parameter：
        ----------
            Obs - 要识别的文本
        Return:
        -------
            None
        Author:
        -------
            Ming Chen
        Modify:
        -------
            2019-11-27 20:53:23
        """
        T = len(Obs)
        path = self.viterbi(Obs)
        for i in range(T):
            print(Obs[i]+self.idx2tag[path[i]]+'_|', end='')
        print()
        print()

In [3]:
#创建一个HMM模型
model = HMM_BIO_method1()

In [4]:
#训练
# model.train('corpus/corpus_part0.txt')
model.train('corpus/corpus_1.txt')

开始训练数据：
参数统计结果(A,PI)
A:[[  0.  17.   0.]
 [  0. 122.  10.]
 [ 10.   0. 664.]]
PI:[ 7.  0. 88.]
训练参数结果(A,B,PI)
A:[[-2.33091723e+02  0.00000000e+00 -2.33091723e+02]
 [-2.35141311e+02 -7.87808779e-02 -2.58021683e+00]
 [-4.21064502e+00 -2.36771739e+02 -1.49479614e-02]]
B:[[-233.09172264 -233.09172264 -233.09172264 ... -233.09172264
  -233.09172264 -233.09172264]
 [-235.19298323 -235.19298323 -235.19298323 ... -235.19298323
  -235.19298323 -235.19298323]
 [-236.89445586 -236.89445586 -236.89445586 ... -236.89445586
  -236.89445586 -236.89445586]]
PI:[-2.60796674e+00 -2.34812386e+02 -7.65400771e-02]
训练参数已保存在 param/BIO/method1/A.csv



In [5]:
#从训练好的模型加载参数
model.load_Param()

训练参数(A,B,PI)已从 param/BIO/method1/A.csv 中读取成功



In [6]:
s='家乐福'
print(model.predict(s))

家O_|乐B-ORG_|福I-ORG_|

None


In [7]:
#进行预测(copus_part0_others_method1)
sentences=['龙川县博物馆馆藏文物','林徽因什么理由拒绝了徐志摩而选择梁思成变为终身伴侣?谢娜为李浩菲澄清网络谣言，之后她的两个行为给自己加分','王心雷编写了一个代码',
      '王康在哈尔滨出门打电动被张舒帆当场逮捕','北京抖音信息服务有限公司今天出台了一个新的政策','中国招标网发布了一个新的招标',
       '我真的好饿','中标人:遂宁高新区蜜感内衣店','中国是个好地方','宁夏最高气温20度','恭喜安居区安居镇有家超市昨天开业了',
           '家乐福是个好地方','家乐福','家乐福超市']
for sentence in sentences:
    model.predict(sentence)
    

龙B-ORG_|川I-ORG_|县I-ORG_|博I-ORG_|物I-ORG_|馆I-ORG_|馆I-ORG_|藏O_|文O_|物O_|

林B-ORG_|徽I-ORG_|因I-ORG_|什I-ORG_|么I-ORG_|理I-ORG_|由I-ORG_|拒I-ORG_|绝I-ORG_|了I-ORG_|徐I-ORG_|志O_|摩O_|而O_|选O_|择B-ORG_|梁I-ORG_|思I-ORG_|成O_|变B-ORG_|为I-ORG_|终I-ORG_|身I-ORG_|伴I-ORG_|侣I-ORG_|?I-ORG_|谢I-ORG_|娜I-ORG_|为I-ORG_|李I-ORG_|浩I-ORG_|菲I-ORG_|澄I-ORG_|清I-ORG_|网I-ORG_|络I-ORG_|谣I-ORG_|言I-ORG_|，I-ORG_|之O_|后O_|她O_|的O_|两O_|个O_|行O_|为O_|给O_|自O_|己O_|加O_|分O_|

王B-ORG_|心I-ORG_|雷I-ORG_|编O_|写O_|了O_|一O_|个O_|代O_|码O_|

王O_|康B-ORG_|在I-ORG_|哈I-ORG_|尔I-ORG_|滨I-ORG_|出O_|门B-ORG_|打I-ORG_|电I-ORG_|动I-ORG_|被I-ORG_|张I-ORG_|舒I-ORG_|帆I-ORG_|当I-ORG_|场I-ORG_|逮I-ORG_|捕I-ORG_|

北B-ORG_|京I-ORG_|抖I-ORG_|音I-ORG_|信O_|息O_|服O_|务B-ORG_|有I-ORG_|限I-ORG_|公I-ORG_|司I-ORG_|今I-ORG_|天I-ORG_|出O_|台O_|了O_|一O_|个O_|新O_|的O_|政B-ORG_|策I-ORG_|

中O_|国B-ORG_|招I-ORG_|标I-ORG_|网I-ORG_|发I-ORG_|布O_|了O_|一O_|个O_|新B-ORG_|的I-ORG_|招I-ORG_|标I-ORG_|

我B-ORG_|真I-ORG_|的O_|好B-ORG_|饿I-ORG_|

中O_|标O_|人O_|:O_|遂B-ORG_|宁I-ORG_|高I-ORG_|新I-ORG_|区O_|蜜O_|感O_|内O_|衣B-ORG_|店I-ORG_|

中O_|国B-ORG_|是I-ORG_|个O_|